In [ ]:
# Copyright 2025 Google LLC
# Copyright Richard He Modified on 2025
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
# Original notebook source from https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/vision/getting-started/virtual_try_on.ipynb

In [ ]:
%pip install --upgrade --quiet google-genai

In [ ]:
import typing
import urllib.request

import IPython.display
from PIL import Image as PIL_Image
from PIL import ImageOps as PIL_ImageOps
from google import genai
from google.genai.types import (
    GenerateImagesConfig,
    Image,
    ProductImage,
    RecontextImageConfig,
    RecontextImageSource,
)
import matplotlib.image as img
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import os

PROJECT_ID = "rocketech-de-pgcp-sandbox"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
LOCATION = 'europe-west4' # @param {type: "string", placeholder: "[location]", isTemplate: true}
virtual_try_on_model = "virtual-try-on-preview-08-04"
image_generation = "imagen-4.0-generate-001"

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

In [ ]:
# Function to display media in the notebook

def display_image(
    image,
    max_width: int = 400,
    max_height: int = 400,
) -> None:
    pil_image = typing.cast(PIL_Image.Image, image._pil_image)
    if pil_image.mode != "RGB":
        # RGB is supported by all Jupyter environments (e.g. RGBA is not yet)
        pil_image = pil_image.convert("RGB")
    image_width, image_height = pil_image.size
    if max_width < image_width or max_height < image_height:
        # Resize to display a smaller notebook image
        pil_image = PIL_ImageOps.contain(pil_image, (max_width, max_height))
    IPython.display.display(pil_image)


def display_local_image(
    images: list[str],
) -> None:
    fig, axes = plt.subplots(1, len(images), figsize=(12, 6))
    if len(images) == 1:
        axes = np.array([axes])
    for i, ax in enumerate(axes):
        image = img.imread(images[i])
        ax.imshow(image)
        ax.axis("off")
    plt.show()

In [ ]:
# Function to recontext the image (virtrual try on in action)

def virtual_try_on(person_image: Image, product_images: Image) -> Image:
  response = client.models.recontext_image(
    model=virtual_try_on_model,
    source=RecontextImageSource(
        person_image=person_image,
        product_images=[
            ProductImage(product_image=product_images)
        ],
    ),
    config=RecontextImageConfig(
        base_steps=32,
        number_of_images=1,
        safety_filter_level="BLOCK_LOW_AND_ABOVE",
        person_generation="ALLOW_ADULT",
    ),
  )

  return response.generated_images[0].image

In [ ]:
# Function to generate image using the Imagen model, only generate one at a time for now to keep it simple

def generate_image(prompt: str, no_human=True) -> Image:

  image = client.models.generate_images(
    model=image_generation,
    prompt=prompt,
    config=GenerateImagesConfig(
        number_of_images=1,
        image_size="1K", # 2k is the max resolution supported now
        safety_filter_level="BLOCK_LOW_AND_ABOVE", # safest setting, see more safety level here https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-filters#how-to-configure
        person_generation="ALLOW_ADULT",
    ),
  )

  return image.generated_images[0].image

In [ ]:
# Human model samples with matching clothing to use virtual try on prompts

sample_sets = [
  {
    "id": 1,
    "human_model_prompt": "A full-body, front-view photograph of a woman standing in a professional photo studio with a plain white background.",
    "clothing_to_try": [
      "A flat-lay of a white t-shirt with a tree shaped logo at the front",
      "A neatly folded pair of plain dark-wash skinny jeans.",
      "A top-down shot of a pair of simple red ballet flats, with no visible branding."
    ]
  },
  {
    "id": 2,
    "human_model_prompt": "A full-body, side-view photograph of a man standing in a studio with a neutral gray seamless paper background.",
    "clothing_to_try": [
      "A crisp white oxford button-down shirt, carefully folded, with a small, discreet text logo 'Origin' embroidered on the cuff.",
      "A pair of plain, tailored gray wool trousers, laid flat.",
      "A profile shot of a single polished black leather oxford shoe against a white background."
    ]
  },
  {
    "id": 3,
    "human_model_prompt": "A three-quarter view, full-body photograph of a woman posing in a photography studio against a soft beige backdrop.",
    "clothing_to_try": [
      "A plain forest green cashmere V-neck sweater, laid flat on a white surface.",
      "A black A-line knee-length corduroy skirt, displayed on a ghost mannequin, featuring a small embroidered logo of a fox near the hem.",
      "A pair of plain black leather knee-high boots with a flat sole, standing upright against a plain background."
    ]
  },
  {
    "id": 4,
    "human_model_prompt": "A full-body, back-view photograph of a man in a studio setting with a light gray background.",
    "clothing_to_try": [
      "A studio product shot of a plain charcoal gray bomber jacket with a silver zipper.",
      "A flat-lay of a pair of straight-leg black denim jeans.",
      "A three-quarter view of a pair of clean white leather court sneakers with a simple text-based logo 'Apex' embossed on the side."
    ]
  },
  {
    "id": 5,
    "human_model_prompt": "A low-angle, full-body photograph of a woman posing in a studio with a clean white cyclorama wall.",
    "clothing_to_try": [
      "A flat-pack image of a vibrant yellow sleeveless blouse with a tie-neck, showcasing an intricate, multi-colored image-based logo of a peacock feather on the collar.",
      "A pair of plain white wide-leg capri pants, neatly folded.",
      "A pair of plain tan espadrille wedge sandals, shot from a slightly elevated angle."
    ]
  },
  {
    "id": 6,
    "human_model_prompt": "A high-angle, three-quarter view, full-body photograph of a man in a photo studio with a dark charcoal background.",
    "clothing_to_try": [
      "An olive green utility shirt with two button-flap chest pockets, laid flat and showing no logos.",
      "A pair of beige cargo shorts with a text-based logo 'Nomad' printed subtly above the back pocket.",
      "A close-up, detailed shot of a pair of brown suede hiking boots with an image-based logo of a mountain peak stamped onto the heel."
    ]
  },
  {
    "id": 7,
    "human_model_prompt": "A high-resolution, full-body, head-on photograph of a woman standing in a brightly lit professional photography studio wearing a dress. The backdrop is clean gray paper.",
    "clothing_to_try": [
      "A studio photograph of a floor-length black silk evening gown with a thigh-high slit, displayed on a ghost mannequin.",
      "A product shot of a deep burgundy velvet mermaid gown with an off-the-shoulder neckline and a dramatic flared hem, presented on a ghost mannequin to highlight its silhouette.",
      "A pair of strappy, high-heeled sandals in a metallic silver finish, featuring a small, intricate image-based logo of a crescent moon on the sole."
    ]
  },
  {
    "id": 8,
    "human_model_prompt": "A static, full-body (including shoes), direct front-on photograph of 3 women in a studio with even lighting against a white background.",
    "clothing_to_try": [
      "A professional single-button black blazer on a mannequin, with a small, text-based logo 'Elysian' on the inner lining.",
      "A pair of plain, matching black tailored trousers, hung on a rack.",
      "A pair of nude-colored pointed-toe pumps, shot from a low angle."
    ]
  },
  {
    "id": 9,
    "human_model_prompt": "A full-body, side-view photograph of a man leaning against a prop block in a studio with a textured gray background.",
    "clothing_to_try": [
      "A flat-lay of a burgundy crewneck sweatshirt featuring a complex, yellow text-based logo in a serif font that reads 'Northbound Collective'.",
      "A pair of light-wash relaxed-fit jeans with a torn knee, laid flat.",
      "A pair of classic brown suede desert boots with no visible logos."
    ]
  },
  {
    "id": 10,
    "human_model_prompt": "A dynamic, full-body (including shoes) photograph from a back three-quarter angle of a non-binary person, shot in a studio with a sky-blue background.",
    "clothing_to_try": [
      "A plain multi-colored windbreaker jacket in purple, teal, and yellow.",
      "A pair of black nylon track pants featuring a recurring image-based logo of a simplified lightning bolt down the side stripe.",
      "A pair of black high-top sneakers with reflective details, shot under dramatic studio lighting."
    ]
  }
]

In [ ]:
# Virtual try on with one item applied at a time, total 3. Applying multiple items at the same time is currently not supported.

def try_it(prompt_index):
  human_model = generate_image(sample_sets[prompt_index]['human_model_prompt'], no_human=False)
  display_image(human_model)

  for i in sample_sets[prompt_index]['clothing_to_try']:
    item = generate_image(i)
    print("Displaying generated item...")
    display_image(item)
    human_model = virtual_try_on(human_model, item) # replacing human_model so all of the items can be tried.
    print("Displaying virtrual try on result...")
    display_image(human_model)

In [ ]:
try_it(0)

In [ ]:
try_it(2)

In [ ]:
try_it(8)

In [ ]:
try_it(9)

In [ ]:
try_it(7)